In [1]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [2]:
# read in with pandas
winter1 = pd.read_csv ('rut20-21data.txt')
winter2 = pd.read_csv ('rut21-22data.txt')
winter3 = pd.read_csv ('rut22-23data.txt')

In [3]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [4]:
# mask out days without precipitation
codes = ['GR','GS','IC','PL','PY','RA','SG','SH','SN','SQ','TS','UP']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [5]:
precip_days2

,station,datetime,precip(mm),wxcodes
0,RUT,2021-12-01 00:19:00,T,-SN BR
1,RUT,2021-12-01 00:37:00,T,-SN BR
2,RUT,2021-12-01 00:56:00,T,-SN BR
3,RUT,2021-12-01 01:13:00,T,-SN BR
4,RUT,2021-12-01 01:41:00,0.25,-SN BR
...,...,...,...,...
2710,RUT,2022-02-27 22:45:00,T,-SN
2711,RUT,2022-02-27 22:50:00,T,-SN
2712,RUT,2022-02-27 22:56:00,T,-SN
2713,RUT,2022-02-27 23:15:00,T,-SN


In [6]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_4480\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_4480\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_4480\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [7]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_4480\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_4480\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_4480\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [8]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 56]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 56]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 56]

In [9]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [10]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [11]:
rut = LocationInfo(
    'Rutland', 'Vermont', 
    latitude=43.6106,
    longitude=-72.9726, 
    timezone='America/New_York'
)

rut

LocationInfo(name='Rutland', region='Vermont', timezone='America/New_York', latitude=43.6106, longitude=-72.9726)

In [12]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(rut.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [13]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(rut.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [14]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(rut.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [15]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 07:06:09  2020-12-01 16:15:55  2020-12-01
1   2020-12-02 07:07:14  2020-12-02 16:15:37  2020-12-02
2   2020-12-03 07:08:18  2020-12-03 16:15:22  2020-12-03
3   2020-12-04 07:09:20  2020-12-04 16:15:10  2020-12-04
4   2020-12-05 07:10:21  2020-12-05 16:15:00  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:36:24  2021-02-24 17:34:17  2021-02-24
86  2021-02-25 06:34:47  2021-02-25 17:35:35  2021-02-25
87  2021-02-26 06:33:09  2021-02-26 17:36:53  2021-02-26
88  2021-02-27 06:31:31  2021-02-27 17:38:10  2021-02-27
89  2021-02-28 06:29:51  2021-02-28 17:39:28  2021-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 07:05:53  2021-12-01 16:15:59  2021-12-01
1   2021-12-02 07:06:59  2021-12-02 16:15:41  2021-12-02
2   2021-12-03 07:08:03  2021-12-03 16:15:25  2021-12-03
3   2021-12-04 07:09:05  2021-12-04 16:15:12  2021-12-04
4   2021-12-05 07:10:06  2021-12-05 16:15:02  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:36:48  2022-02-24 17:33:58  2022-02-24
86  2022-02-25 06:35:11  2022-02-25 17:35:16  2022-02-25
87  2022-02-26 06:33:33  2022-02-26 17:36:34  2022-02-26
88  2022-02-27 06:31:55  2022-02-27 17:37:52  2022-02-27
89  2022-02-28 06:30:15  2022-02-28 17:39:09  2022-02-28

[90 rows x 3 columns]


In [17]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 07:05:38  2022-12-01 16:16:03  2022-12-01
1   2022-12-02 07:06:43  2022-12-02 16:15:45  2022-12-02
2   2022-12-03 07:07:47  2022-12-03 16:15:29  2022-12-03
3   2022-12-04 07:08:50  2022-12-04 16:15:15  2022-12-04
4   2022-12-05 07:09:52  2022-12-05 16:15:04  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:37:11  2023-02-24 17:33:39  2023-02-24
86  2023-02-25 06:35:34  2023-02-25 17:34:57  2023-02-25
87  2023-02-26 06:33:57  2023-02-26 17:36:15  2023-02-26
88  2023-02-27 06:32:18  2023-02-27 17:37:33  2023-02-27
89  2023-02-28 06:30:39  2023-02-28 17:38:50  2023-02-28

[90 rows x 3 columns]


In [18]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

C:\Users\alyha\AppData\Local\Temp\ipykernel_4480\762931898.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['date'] = precip_days2['datetime'].dt.date


In [19]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,RUT,2020-12-01 00:56:00,0.0000,-RA,2020-12-01,2020-12-01 07:06:09,2020-12-01 16:15:55
1,RUT,2020-12-01 17:56:00,0.0000,-RA,2020-12-01,2020-12-01 07:06:09,2020-12-01 16:15:55
2,RUT,2020-12-02 00:56:00,0.0000,-RA,2020-12-02,2020-12-02 07:07:14,2020-12-02 16:15:37
3,RUT,2020-12-02 01:56:00,0.0000,-RA,2020-12-02,2020-12-02 07:07:14,2020-12-02 16:15:37
4,RUT,2020-12-02 03:56:00,0.0000,-RA,2020-12-02,2020-12-02 07:07:14,2020-12-02 16:15:37
...,...,...,...,...,...,...,...
561,RUT,2021-02-27 12:56:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:31:31,2021-02-27 17:38:10
562,RUT,2021-02-27 13:56:00,0.0001,UP,2021-02-27,2021-02-27 06:31:31,2021-02-27 17:38:10
563,RUT,2021-02-27 14:56:00,0.0001,-RA,2021-02-27,2021-02-27 06:31:31,2021-02-27 17:38:10
564,RUT,2021-02-27 15:56:00,0.0001,-RA,2021-02-27,2021-02-27 06:31:31,2021-02-27 17:38:10


In [20]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,RUT,2021-12-01 00:56:00,0.0001,-SN BR,2021-12-01,2021-12-01 07:05:53,2021-12-01 16:15:59
1,RUT,2021-12-01 01:56:00,0.2500,-SN BR,2021-12-01,2021-12-01 07:05:53,2021-12-01 16:15:59
2,RUT,2021-12-01 02:56:00,0.0001,-SN,2021-12-01,2021-12-01 07:05:53,2021-12-01 16:15:59
3,RUT,2021-12-01 03:56:00,0.0001,-SN,2021-12-01,2021-12-01 07:05:53,2021-12-01 16:15:59
4,RUT,2021-12-01 04:56:00,0.0001,-SN BR,2021-12-01,2021-12-01 07:05:53,2021-12-01 16:15:59
...,...,...,...,...,...,...,...
468,RUT,2022-02-27 19:56:00,0.0001,-SN,2022-02-27,2022-02-27 06:31:55,2022-02-27 17:37:52
469,RUT,2022-02-27 20:56:00,0.0001,-SN,2022-02-27,2022-02-27 06:31:55,2022-02-27 17:37:52
470,RUT,2022-02-27 21:56:00,0.0001,-SN,2022-02-27,2022-02-27 06:31:55,2022-02-27 17:37:52
471,RUT,2022-02-27 22:56:00,0.0001,-SN,2022-02-27,2022-02-27 06:31:55,2022-02-27 17:37:52


In [21]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,RUT,2022-12-01 06:56:00,0.0001,-SN,2022-12-01,2022-12-01 07:05:38,2022-12-01 16:16:03
1,RUT,2022-12-01 08:56:00,0.0001,UP,2022-12-01,2022-12-01 07:05:38,2022-12-01 16:16:03
2,RUT,2022-12-01 09:56:00,0.0001,-SN,2022-12-01,2022-12-01 07:05:38,2022-12-01 16:16:03
3,RUT,2022-12-01 10:56:00,0.0001,-SN BR,2022-12-01,2022-12-01 07:05:38,2022-12-01 16:16:03
4,RUT,2022-12-01 11:56:00,0.0001,-SN,2022-12-01,2022-12-01 07:05:38,2022-12-01 16:16:03
...,...,...,...,...,...,...,...
549,RUT,2023-02-27 06:56:00,0.2500,-SN BR,2023-02-27,2023-02-27 06:32:18,2023-02-27 17:37:33
550,RUT,2023-02-27 07:56:00,0.0001,-SN,2023-02-27,2023-02-27 06:32:18,2023-02-27 17:37:33
551,RUT,2023-02-27 10:56:00,0.0001,-SN,2023-02-27,2023-02-27 06:32:18,2023-02-27 17:37:33
552,RUT,2023-02-27 11:56:00,0.0001,-SN,2023-02-27,2023-02-27 06:32:18,2023-02-27 17:37:33


In [22]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
7,RUT,2020-12-02 12:56:00,0.0000,-RA,2020-12-02,2020-12-02 07:07:14,2020-12-02 16:15:37
8,RUT,2020-12-02 15:56:00,0.0000,-SN,2020-12-02,2020-12-02 07:07:14,2020-12-02 16:15:37
20,RUT,2020-12-05 13:56:00,0.0000,-RA,2020-12-05,2020-12-05 07:10:21,2020-12-05 16:15:00
21,RUT,2020-12-05 14:56:00,0.0000,-RA,2020-12-05,2020-12-05 07:10:21,2020-12-05 16:15:00
22,RUT,2020-12-05 15:56:00,0.0000,-RA BR,2020-12-05,2020-12-05 07:10:21,2020-12-05 16:15:00
...,...,...,...,...,...,...,...
560,RUT,2021-02-27 11:56:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:31:31,2021-02-27 17:38:10
561,RUT,2021-02-27 12:56:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:31:31,2021-02-27 17:38:10
562,RUT,2021-02-27 13:56:00,0.0001,UP,2021-02-27,2021-02-27 06:31:31,2021-02-27 17:38:10
563,RUT,2021-02-27 14:56:00,0.0001,-RA,2021-02-27,2021-02-27 06:31:31,2021-02-27 17:38:10


In [23]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_4480\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_4480\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_4480\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [24]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
7,RUT,2020-12-02 12:56:00,0.0000,-RA,2020-12-02,2020-12-02 07:07:14,2020-12-02 16:15:37,2020-12-02 08:07:14,2020-12-02 15:15:37
20,RUT,2020-12-05 13:56:00,0.0000,-RA,2020-12-05,2020-12-05 07:10:21,2020-12-05 16:15:00,2020-12-05 08:10:21,2020-12-05 15:15:00
21,RUT,2020-12-05 14:56:00,0.0000,-RA,2020-12-05,2020-12-05 07:10:21,2020-12-05 16:15:00,2020-12-05 08:10:21,2020-12-05 15:15:00
31,RUT,2020-12-06 09:56:00,0.0000,-SN,2020-12-06,2020-12-06 07:11:20,2020-12-06 16:14:52,2020-12-06 08:11:20,2020-12-06 15:14:52
32,RUT,2020-12-06 10:56:00,0.0000,-SN,2020-12-06,2020-12-06 07:11:20,2020-12-06 16:14:52,2020-12-06 08:11:20,2020-12-06 15:14:52
...,...,...,...,...,...,...,...,...,...
560,RUT,2021-02-27 11:56:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:31:31,2021-02-27 17:38:10,2021-02-27 07:31:31,2021-02-27 16:38:10
561,RUT,2021-02-27 12:56:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:31:31,2021-02-27 17:38:10,2021-02-27 07:31:31,2021-02-27 16:38:10
562,RUT,2021-02-27 13:56:00,0.0001,UP,2021-02-27,2021-02-27 06:31:31,2021-02-27 17:38:10,2021-02-27 07:31:31,2021-02-27 16:38:10
563,RUT,2021-02-27 14:56:00,0.0001,-RA,2021-02-27,2021-02-27 06:31:31,2021-02-27 17:38:10,2021-02-27 07:31:31,2021-02-27 16:38:10


In [25]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [26]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3


In [27]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.18518177083333331
2021-2022: 0.11804203821656051
2022-2023: 0.24912366863905325


In [28]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,RUT,2020-12-01 00:56:00,0.0000,-RA,2020-12-01,2020-12-01 07:06:09,2020-12-01 16:15:55
1,RUT,2020-12-01 17:56:00,0.0000,-RA,2020-12-01,2020-12-01 07:06:09,2020-12-01 16:15:55
2,RUT,2020-12-02 00:56:00,0.0000,-RA,2020-12-02,2020-12-02 07:07:14,2020-12-02 16:15:37
3,RUT,2020-12-02 01:56:00,0.0000,-RA,2020-12-02,2020-12-02 07:07:14,2020-12-02 16:15:37
4,RUT,2020-12-02 03:56:00,0.0000,-RA,2020-12-02,2020-12-02 07:07:14,2020-12-02 16:15:37
...,...,...,...,...,...,...,...
554,RUT,2021-02-23 23:56:00,0.0001,UP,2021-02-23,2021-02-23 06:38:01,2021-02-23 17:32:58
555,RUT,2021-02-24 02:56:00,0.0001,-RA,2021-02-24,2021-02-24 06:36:24,2021-02-24 17:34:17
556,RUT,2021-02-25 02:56:00,0.0001,-RA,2021-02-25,2021-02-25 06:34:47,2021-02-25 17:35:35
557,RUT,2021-02-25 03:56:00,0.0001,-RA,2021-02-25,2021-02-25 06:34:47,2021-02-25 17:35:35


In [29]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_4480\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_4480\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_4480\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [30]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [31]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [32]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.04706410256410255
2021-2022: 0.12112761194029852
2022-2023: 0.13843289473684212
